Problems:
1.should achieve model save then keep training (online learning)
2.early stopping not work
3.any like Alexnet for word?
4.dont know how to adjust model, should consider using AWS?
5.textgenrnn

In [96]:
# Importing Libraries
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow
from tensorflow.keras.layers import Embedding,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,GRU
from tensorflow.keras.layers import Dense

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ouyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
train = pd.read_csv("train.csv")
predict  = pd.read_csv("test.csv")

In [98]:
#temp
train = train[1:100]

In [99]:
# Displaying rows and columns in dataset
print("There are {} number of rows and {} number of columns for training.".format(train.shape[0],train.shape[1]))
print("There are {} number of rows and {} number of columns for testing.".format(predict.shape[0],test.shape[1]))

There are 99 number of rows and 5 number of columns for training.
There are 5200 number of rows and 5 number of columns for testing.


In [100]:
def handle_nan(train_data,predict_data):
    '''Input: Data to the function containing Nan values.
       Output : Cleaned data containing no Nan values.
       Function: Cleaning Nan values.
     '''
    train = train_data.fillna(" ")
    predict  = predict_data.fillna(" ")
    return train,predict

train,predict = handle_nan(train,predict)

In [101]:
# Creating a variable "merged" by merging columns "title" and "author"
train["merged"] = train["title"]+" "+train["text"]
predict["merged"]  = predict["title"]+" "+predict["text"]

In [102]:
# Seperating Independent and dependent features
X = train.drop(columns=['label'],axis=1)
y = train['label']

In [103]:
# Reset Index
X.reset_index(inplace=True)

predict.reset_index(inplace=True)

# Data Pre-processing
**In Data Pre-processing following steps are followed:** 
**1. Firstly, all the sequences except english characters are removed from the string.**
**2. Next, to avoid false predictions or ambiguity with upper and lowercase, all the characters in strings are converted    to lowercase.**
**3. Next, all the sentences are tokenized into words.**
**4. To facilitate fast processing, stemming is applied to the tokenized words.**
**5. Next, words are joined together and stored in the corpus.**

**Note: In this tutorial, we have used "merged" column for classification task. Also, the loop inside the function runs over all the examples in the merged column.**

In [104]:
from nltk.stem.porter import PorterStemmer

def normalize(data):
    normalized = []
    for i in data:
        i = i.lower()
        # get rid of urls
        i = re.sub('https?://\S+|www\.\S+', '', i)
        # get rid of non words and extra spaces
        i = re.sub('\\W', ' ', i)
        i = re.sub('\n', '', i)
        i = re.sub(' +', ' ', i)
        i = re.sub('^ ', '', i)
        i = re.sub(' $', '', i)
        normalized.append(i)
    return normalized

X["merged"] = normalize(X["merged"])
predict["merged"] = normalize(predict["merged"])

In [105]:
# Performing data preprocessing on column 'title'
ps = PorterStemmer()
def perform_preprocess(data):
    '''Input: Data to be processed
       Output: Preprocessed data
    '''
    corpus = []
    for i in range(0,len(data)):
        review = re.sub('[^a-zA-Z]',' ',data['merged'][i])
        review = review.lower()
        review = review.split()
        review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [106]:
train_corpus = perform_preprocess(X)
#predict_corpus  = perform_preprocess(predict)

In [107]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
train_corpus = v.fit_transform(X["merged"])

In [115]:
len(train_corpus.toarray())

99

In [117]:
len(train_corpus.toarray()[0])

12022

**Below code converts the pre-processed words to one-hot vectors in the range of vocabulary size=5000. This is done to obtain numerical feature matrix**

In [62]:
len(one_hot_train)

NameError: name 'one_hot_train' is not defined

In [ ]:
len(one_hot_train[0])

374

**Below code creates an embedding layer which applies "pre" padding to the one-hot encoded features with sentence length = 20. Padding is applied so that the length of every sequence in the dataset should be same.**

In [63]:
# Embedding Representation 
sent_length = 500
embedd_docs_train = pad_sequences(one_hot_train,padding='pre',maxlen=sent_length)
#embedd_docs_test  = pad_sequences(one_hot_test,padding='pre',maxlen=sent_length)

NameError: name 'one_hot_train' is not defined

In [ ]:
len(embedd_docs_train[0])

500

In [ ]:
# Converting Embedding repr. to array
x_final = np.array(embedd_docs_train)
y_final = np.array(y)

#x_test_final = np.array(embedd_docs_test)

In [ ]:
# Dimensions of prev. array repr.
x_final.shape, y_final.shape#, x_test_final.shape

((99, 500), (99,))

**Dividing the dataset into training and testing data.**

In [118]:
from sklearn.model_selection import train_test_split

In [120]:
x_train, x_test, y_train, y_test = train_test_split(train_corpus, y, test_size=0.1, random_state=42, stratify = y)

In [121]:
x_train.shape

(89, 12022)

In [122]:
y_train.shape

(89,)

**LSTM**

**In this model, 1.) The value for embedding feature vectors = 40 which are target feature vectors for the embedding layer. 2.) Single LSTM Layer with 100 nodes are used. 3.)Dense Layer with 1 neuron and sigmoid activation function is used since, this is a binary classification problem. 4) Dropout technique is used to avoid overfiiting and adam optimizer is used for optimizing the loss function.**

In [123]:
# Creating the LSTM Model for prediction
embedding_feature_vector = 40
def create_model():
    model = Sequential()
    model.add(Embedding(embedding_feature_vector))
    model.add(Dropout(0.3))
    model.add(LSTM(100))
    model.add(Dropout(0.3))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    return model

In [124]:
model = create_model()
model.summary()

TypeError: __init__() missing 1 required positional argument: 'output_dim'

In [ ]:
# Training the model
model.fit(x_train,y_train,validation_split=0.1,epochs=100,batch_size=64)

Epoch 1/100
2/2 [==============================] - 0s 92ms/step - loss: 3.4001e-04 - accuracy: 1.0000 - val_loss: 1.5545 - val_accuracy: 0.7778
Epoch 2/100
2/2 [==============================] - 0s 71ms/step - loss: 2.2929e-04 - accuracy: 1.0000 - val_loss: 1.5502 - val_accuracy: 0.7778
Epoch 3/100
2/2 [==============================] - 0s 62ms/step - loss: 3.3749e-04 - accuracy: 1.0000 - val_loss: 2.1648 - val_accuracy: 0.6667
Epoch 4/100
2/2 [==============================] - 0s 74ms/step - loss: 3.0621e-04 - accuracy: 1.0000 - val_loss: 2.0641 - val_accuracy: 0.6667
Epoch 5/100
2/2 [==============================] - 0s 72ms/step - loss: 3.6601e-04 - accuracy: 1.0000 - val_loss: 2.0631 - val_accuracy: 0.6667
Epoch 6/100
2/2 [==============================] - 0s 71ms/step - loss: 2.6709e-04 - accuracy: 1.0000 - val_loss: 2.0978 - val_accuracy: 0.5556
Epoch 7/100
2/2 [==============================] - 0s 70ms/step - loss: 3.1608e-04 - accuracy: 1.0000 - val_loss: 2.1442 - val_accuracy:

In [ ]:
model.save('ML_LSTM_v2.model')

INFO:tensorflow:Assets written to: ML_LSTM_v2.model\assets


INFO:tensorflow:Assets written to: ML_LSTM_v2.model\assets


In [ ]:
#model.load_weights('ML_LSTM_v2.model')

In [ ]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 33ms/step - loss: 1.5595 - accuracy: 0.7000


[1.5595232248306274, 0.699999988079071]

In [ ]:
x_test_final.shape

In [1]:
# Making Predictions on test data
predictions_test = pd.DataFrame(model.predict(x_test_final))
test_id = pd.DataFrame(test["id"])

submission = pd.concat([test_id, predictions_test],axis=1)
submission.columns = ["id","LSTM_label"]
submission.to_csv("ML_LSTM_v2_Submission.csv",index=False)

NameError: name 'pd' is not defined

In [ ]:
predictions_test

In [ ]:
submission.shape

In [ ]:
submission.head()